In [ ]:
class Mail:
    def __init__(self,From,date_time,msg_id,to,body,subject="",replay_to="",cc=[],bcc=[]):  
                self.From = From
                self.date_time = date_time
                self.msg_id= msg_id
                self.subject = subject
                self.replay_to = replay_to
                self.to = to
                self.body = body
                self.cc = cc
                self.bcc = bcc      

In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from pprint import pprint
import re
from datetime import datetime
import base64
#from bs4 import BeautifulSoup
#import mail

class GmailConnexion:
    credentials = None
    def __init__(self,labelIds='INBOX', maxResults = 100):
        self.labelIds = labelIds
        self.maxResults = maxResults

    def ckeck_credentials(self):
        if not self.credentials:
            SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            self.credentials = flow.run_local_server(port=0)
        if self.credentials:
            if self.credentials.expired:
                self.credentials.refresh(Request())
        self.service = build('gmail', 'v1', credentials=self.credentials)

    def readInfoUser(self):
        userInfo = self.service.users().getProfile(userId='me').execute()
        pprint("Details of users mailing is \n %s" % (userInfo))
    
    def get_emails(self,s,unique=False):
        regEmails = re.compile(r'\"?(?P<EMAIL_TITLE>[\w\.\-_]+)?\"?\s?<(?P<EMAIL>[\w\._\-]+@[\w\._\-]+)>',flags=re.I)
        match = re.search(regEmails, s)
        if match:
        #     print("".join([i for i in match.groups() if i]))
            pprint(match.groupdict())
        #if unique:
        #    re.search(regex, s, re.M|re.I)
        #else:
        #    return regEmails.findall(s)

        
    def read(self):
            emails = []
            # Call the Gmail API
            results = self.service.users().messages().list(userId='me',labelIds =self.labelIds,maxResults=self.maxResults).execute()
            messages = results.get('messages', [])
            if not messages:
                pprint('No labels found.')
            else:
                for num,message in enumerate(messages):
                    temp={}
                    temp['msg'] = self.service.users().messages().get(userId='me', id=message['id']).execute()
                    #pprint(self.msg)
                    temp['payld'] = temp['msg']['payload']
                    temp['dict'] = {i['name']:i['value'] for i in temp['msg']['payload']['headers']}
                    temp['From'] = self.get_emails(temp['dict']['From'],unique=True)
                    temp['to'] = self.get_emails(temp['dict']['To'])
                    temp['subject'] = temp['dict']['Subject']
                    temp['date_time'] = temp['dict']['Date']
                    msg_id = message['id']
                    if temp['payld']['parts']: 
                        temp['email_parts'] = temp['payld']['parts'] 
                    temp['part_one']  = temp['email_parts'][0]
                    temp['part_body'] = temp['part_one']['body']
                    temp['part_data'] = temp['part_body'].get('data',"") 
                    temp['clean_one'] = temp['part_data'].replace("-","+") 
                    temp['clean_one'] = temp['clean_one'].replace("_","/") 
                    temp['clean_two'] = base64.b64decode (bytes(temp['clean_one'], 'UTF-8'))
                    temp['body'] = temp['clean_two'].decode('utf8')
                    replay_to= ''
                    cc=[]
                    bcc=[]
                    
                    #From,date,msg_id,To,content,subject="",replay_to="",cc=[],bcc=[]
                    current_email = Mail(From = temp['From'],
                                date_time = temp.get('date_time'),
                                msg_id = temp.get('msg_id'),
                                to = temp.get('to',[]),
                                body = temp.get('body'),
                                subject = temp.get('subject'),
                                replay_to = temp.get('replay_to'),
                                cc = temp.get('cc',[]),
                                bcc = temp.get('bcc',[]))
                    current_email.temp=temp
                    emails.append(current_email)
            return emails


In [ ]:
#DEFAULT LABEL 'INBOX' exp (SENT,UNREAD) MAXRESULT '100'
#you can send it within the connexion() method
#this case we take only the first
connect = GmailConnexion('INBOX',1)

In [ ]:
connect.ckeck_credentials()

In [ ]:
connect.readInfoUser()

In [ ]:
emails = connect.read()

In [ ]:
emails